In [1]:
#79############################################################################
#72#####################################################################
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.utils import shuffle
from datetime import datetime

pd.set_option('display.max_columns', None)

In [2]:
filepath = ('C:/Users/andre/OneDrive/Desktop/GitHub/Health-AI/'
            'icr-identify-age-related-conditions/')

train   = pd.read_csv(filepath+'train.csv')
test    = pd.read_csv(filepath+'test.csv')
greeks  = pd.read_csv(filepath+'greeks.csv')
example = pd.read_csv(filepath+'sample_submission.csv')

In [3]:
def prep_df(df):
    # Move categorical col to beginning of df
    cols = list(df.columns)
    cols.remove('EJ')
    # cols.insert(1,'EJ') # Temporarily I will remove this
    df=df[cols]
    
    df=df.set_index('Id').copy(deep=True)

    # Temporarily just fillna
    df.fillna(0,inplace=True)

    # Remove whitespace from column names
    renamer={}
    for col in df.columns:
        renamer[col]=col.strip()
    df.rename(columns=renamer,inplace=True)
    return df
train=prep_df(train)
test=prep_df(test)

In [4]:
foldCount=3

train = shuffle(train, random_state=1)
# Separate into input variables and target variable.
x = train[train.columns[:-1]].copy(deep=True)
y = train[train.columns[-1]].copy(deep=True)
del train

rowsPerFold=len(x)//foldCount
BonusRowsInLastFold=len(x)%foldCount


resultslist=[]
oof_results=[]

for iteration in range(foldCount):
    print('starting iteration',iteration)
   
    # Choosing data to include in this
    firstRow=iteration*rowsPerFold
    if iteration+1==foldCount:
        rowJustPastFold=firstRow+rowsPerFold+BonusRowsInLastFold
    else: rowJustPastFold=firstRow+rowsPerFold
    currentx = pd.concat([x.iloc[:firstRow],x.iloc[rowJustPastFold:]])
    currenty = pd.concat([y.iloc[:firstRow],y.iloc[rowJustPastFold:]])
   
    # Build Model
    model = XGBClassifier(objective='binary:logistic',
                          random_state=1,
                          verbosity=1,
                          use_label_encoder=False,
                          n_jobs=-1,
                          colsample_bytree=0.05,
                          gamma=0.4,
                          learning_rate=.0225,
                          min_child_weight=10,
                          subsample=0.7,
                          n_estimators=5000,
                          max_depth=3,
                          reg_lambda=4)
    model.fit(currentx, currenty, verbose=1)

    # Score Out of fold (oof)
    for predictedVal in model.predict_proba(
        x.iloc[firstRow:rowJustPastFold])[:,1]:
        oof_results.append(predictedVal)

    # Score Test Dataset
    """
    Predict_proba returns a 2 dimensional array, but I will just keep the
    probabilities of class_1. Hence [:,1]
    """
    results = model.predict_proba(test)[:,1]
    resultslist.append(results)


# Average the predictions on the test dataset and store it
results = []
for pos in range(len(resultslist[0])):
    prediction=0
    for listX in range(foldCount):
        prediction+=resultslist[listX][pos]
    prediction=prediction/foldCount
    results.append(prediction)

starting iteration 0


C:\Users\andre\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


starting iteration 1


C:\Users\andre\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


starting iteration 2


C:\Users\andre\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [5]:
submission = test.copy(deep=True)
submission['class_1'] = results
submission['class_0'] = [1 - x for x in results]
submission=submission[['class_0','class_1']]

# Save the out of fold predictions on the train dataset
x['Class']=oof_results
x=x['Class']
x.sort_index(inplace=True)

In [8]:
save_output=False

if save_output:
    outputFileName = 'v2'
    outputFileName += (' ' + str(datetime.today())[:22].replace(':','.'))
    x.to_csv(outputFileName + ' ooftrain.csv')
    submission.to_csv(outputFileName + '.csv')